In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.use('Agg')

In [2]:
# Load training data
train_data = pd.read_pickle('Cleaned_Data/train.pkl')

In [3]:
train_data.head()

,Date,Customer_Code,Customers_Country_Residence,Sex,Age,Date_Acc_Open,New_customer_index,Customer_Seniority,Primary_Customer,Customer_Type_Beginning_Month,...,Mortgage,Pensions1,Loans,Taxes,Credit_card,Securities,Home_Account,Payroll,Pensions2,Direct_Debit
0,2015-01-28,1375586,ES,H,35.0,2015-01-12,0.0,0.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,ES,H,22.0,2012-08-10,0.0,35.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0


In [4]:
# Second month
train_data_mnth = train_data[train_data['Date']==pd.to_datetime('2015-02-28')]

In [5]:
# Introduce lag features
y = ['Saving_Account', 'Guarantees',
       'Current_Account', 'Derivada_Account', 'Payroll_Account',
       'Junior_Account', 'MAS_Particular_Account', 'Particular_Account',
       'Particular_Plus_Account', 'Short_Term_Deposit', 'Medium_Term_Deposit',
       'Long_Term_Deposit', 'E_Account', 'Funds', 'Mortgage', 'Pensions1',
       'Loans', 'Taxes', 'Credit_card', 'Securities', 'Home_Account',
       'Payroll', 'Pensions2', 'Direct_Debit']

In [6]:
# Get the customer products from the previous month
lag_products = train_data[train_data['Date']==pd.to_datetime('2015-01-28')].loc[:,y]
len(lag_products)

615161

In [7]:
# Add prev tag to lag features
lag_products.columns = ['prev_'+ x for x in lag_products.columns]
lag_products.columns

Index(['prev_Saving_Account', 'prev_Guarantees', 'prev_Current_Account',
       'prev_Derivada_Account', 'prev_Payroll_Account', 'prev_Junior_Account',
       'prev_MAS_Particular_Account', 'prev_Particular_Account',
       'prev_Particular_Plus_Account', 'prev_Short_Term_Deposit',
       'prev_Medium_Term_Deposit', 'prev_Long_Term_Deposit', 'prev_E_Account',
       'prev_Funds', 'prev_Mortgage', 'prev_Pensions1', 'prev_Loans',
       'prev_Taxes', 'prev_Credit_card', 'prev_Securities',
       'prev_Home_Account', 'prev_Payroll', 'prev_Pensions2',
       'prev_Direct_Debit'],
      dtype='object')

In [8]:
# Make customer code the index
lag_products.index = train_data[train_data['Date']==pd.to_datetime('2015-01-28')]['Customer_Code']

In [9]:
# Make customer code the index
train_data_mnth.index = train_data_mnth['Customer_Code']

In [10]:
# Left join using customer code
train_data_with_lag_features = pd.concat([train_data_mnth,lag_products],join_axes=[train_data_mnth.index],axis=1)

In [11]:
# Due to new customer some lag features become nans
train_data_with_lag_features.isnull().sum()

Date                                         0
Customer_Code                                0
Customers_Country_Residence                  0
Sex                                          0
Age                                          0
Date_Acc_Open                                0
New_customer_index                           0
Customer_Seniority                           0
Primary_Customer                             0
Customer_Type_Beginning_Month                0
Customer_Relation_Type_Beginning_Month       0
Residence_Index                              0
Foreigner_Index                              0
Channel_Customer_join                        0
Deceased_Index                               0
Address_Type                                 0
Province_Code                                0
Province_Name                                0
Activity_Index                               0
Household_Income                             0
Segmentation                                 0
Saving_Accoun

In [12]:
# Reset the df index
train_data_with_lag_features.reset_index(drop=True,inplace=True)

In [13]:
train_data_with_lag_features.head(5)

,Date,Customer_Code,Customers_Country_Residence,Sex,Age,Date_Acc_Open,New_customer_index,Customer_Seniority,Primary_Customer,Customer_Type_Beginning_Month,...,prev_Mortgage,prev_Pensions1,prev_Loans,prev_Taxes,prev_Credit_card,prev_Securities,prev_Home_Account,prev_Payroll,prev_Pensions2,prev_Direct_Debit
0,2015-02-28,545090,ES,V,67.0,2005-08-02,0.0,120.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
1,2015-02-28,545106,ES,H,40.0,2005-08-30,0.0,120.0,1.0,1.0,...,0,0.0,0,0,1,0,0,0.0,0.0,0
2,2015-02-28,545102,ES,V,46.0,2005-08-02,0.0,120.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,0
3,2015-02-28,545101,ES,V,38.0,2005-08-02,0.0,120.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,1
4,2015-02-28,545158,ES,V,47.0,2005-08-03,0.0,108.0,1.0,1.0,...,0,0.0,1,0,0,0,0,0.0,0.0,0


In [14]:
# Check if the lag features are appended as per the customer codes
train_data[(train_data['Date']==pd.to_datetime('2015-01-28')) & (train_data['Customer_Code']==545101)]

,Date,Customer_Code,Customers_Country_Residence,Sex,Age,Date_Acc_Open,New_customer_index,Customer_Seniority,Primary_Customer,Customer_Type_Beginning_Month,...,Mortgage,Pensions1,Loans,Taxes,Credit_card,Securities,Home_Account,Payroll,Pensions2,Direct_Debit
618008,2015-01-28,545101,ES,V,38.0,2005-08-02,0.0,120.0,1.0,1.0,...,0,0.0,0,0,0,0,0,0.0,0.0,1


In [15]:
lag_features_name = ['prev_Saving_Account', 'prev_Guarantees', 'prev_Current_Account',
       'prev_Derivada_Account', 'prev_Payroll_Account', 'prev_Junior_Account',
       'prev_MAS_Particular_Account', 'prev_Particular_Account',
       'prev_Particular_Plus_Account', 'prev_Short_Term_Deposit',
       'prev_Medium_Term_Deposit', 'prev_Long_Term_Deposit', 'prev_E_Account',
       'prev_Funds', 'prev_Mortgage', 'prev_Pensions1', 'prev_Loans',
       'prev_Taxes', 'prev_Credit_card', 'prev_Securities',
       'prev_Home_Account', 'prev_Payroll', 'prev_Pensions2',
       'prev_Direct_Debit']

In [16]:
# Convert to nan to fill
train_data_with_lag_features[lag_features_name] = train_data_with_lag_features[lag_features_name].astype('float')

In [17]:
# Deal with na values
train_data_with_lag_features[lag_features_name] = train_data_with_lag_features[lag_features_name].fillna(0)

In [18]:
# Convert back to categorical data
train_data_with_lag_features[lag_features_name] = train_data_with_lag_features[lag_features_name].astype('category')

In [19]:
train_data_with_lag_features.isnull().sum()

Date                                      0
Customer_Code                             0
Customers_Country_Residence               0
Sex                                       0
Age                                       0
Date_Acc_Open                             0
New_customer_index                        0
Customer_Seniority                        0
Primary_Customer                          0
Customer_Type_Beginning_Month             0
Customer_Relation_Type_Beginning_Month    0
Residence_Index                           0
Foreigner_Index                           0
Channel_Customer_join                     0
Deceased_Index                            0
Address_Type                              0
Province_Code                             0
Province_Name                             0
Activity_Index                            0
Household_Income                          0
Segmentation                              0
Saving_Account                            0
Guarantees                      

In [20]:
train_data = train_data_with_lag_features

In [21]:
# Columns
train_data.columns

Index(['Date', 'Customer_Code', 'Customers_Country_Residence', 'Sex', 'Age',
       'Date_Acc_Open', 'New_customer_index', 'Customer_Seniority',
       'Primary_Customer', 'Customer_Type_Beginning_Month',
       'Customer_Relation_Type_Beginning_Month', 'Residence_Index',
       'Foreigner_Index', 'Channel_Customer_join', 'Deceased_Index',
       'Address_Type', 'Province_Code', 'Province_Name', 'Activity_Index',
       'Household_Income', 'Segmentation', 'Saving_Account', 'Guarantees',
       'Current_Account', 'Derivada_Account', 'Payroll_Account',
       'Junior_Account', 'MAS_Particular_Account', 'Particular_Account',
       'Particular_Plus_Account', 'Short_Term_Deposit', 'Medium_Term_Deposit',
       'Long_Term_Deposit', 'E_Account', 'Funds', 'Mortgage', 'Pensions1',
       'Loans', 'Taxes', 'Credit_card', 'Securities', 'Home_Account',
       'Payroll', 'Pensions2', 'Direct_Debit', 'prev_Saving_Account',
       'prev_Guarantees', 'prev_Current_Account', 'prev_Derivada_Account',
 

In [22]:
# Numeric features
numeric_cols = ['Age','Customer_Seniority','Household_Income']
# Not categorical features
not_categorical_cols = ['Date','Date_Acc_Open','Customer_Code']
 
categorical = ['Customers_Country_Residence','']

# Y
y = ['Saving_Account', 'Guarantees',
       'Current_Account', 'Derivada_Account', 'Payroll_Account',
       'Junior_Account', 'MAS_Particular_Account', 'Particular_Account',
       'Particular_Plus_Account', 'Short_Term_Deposit', 'Medium_Term_Deposit',
       'Long_Term_Deposit', 'E_Account', 'Funds', 'Mortgage', 'Pensions1',
       'Loans', 'Taxes', 'Credit_card', 'Securities', 'Home_Account',
       'Payroll', 'Pensions2', 'Direct_Debit']

df_numeric = train_data[numeric_cols]
df_categorical = train_data[list(set(train_data.columns)-set(numeric_cols)-set(not_categorical_cols)-set(y))]

In [23]:
# Keep only the base categorical features in the data
to_remove_from_categorical= ['Customers_Country_Residence','Primary_Customer','Customer_Type_Beginning_Month','Channel_Customer_join','Deceased_Index','Address_Type','Province_Code']
df_categorical.drop(to_remove_from_categorical,axis=1,inplace=True)

In [24]:
df_categorical.head(5)

,prev_Taxes,prev_Current_Account,Segmentation,Foreigner_Index,prev_Long_Term_Deposit,prev_Guarantees,prev_Funds,prev_Loans,prev_Saving_Account,Sex,...,Customer_Relation_Type_Beginning_Month,prev_Pensions1,prev_Mortgage,prev_Particular_Account,prev_Payroll_Account,prev_MAS_Particular_Account,New_customer_index,prev_Home_Account,prev_Particular_Plus_Account,prev_Derivada_Account
0,0.0,1.0,02 - PARTICULARES,N,0.0,0.0,0.0,0.0,0.0,V,...,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,02 - PARTICULARES,N,0.0,0.0,0.0,0.0,0.0,H,...,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,02 - PARTICULARES,N,0.0,0.0,0.0,0.0,0.0,V,...,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,02 - PARTICULARES,N,0.0,0.0,0.0,0.0,0.0,V,...,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,02 - PARTICULARES,N,0.0,0.0,0.0,1.0,0.0,V,...,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_categorical = pd.get_dummies(df_categorical)

In [26]:
df_categorical.head(5)

,New_customer_index,prev_Taxes_0.0,prev_Taxes_1.0,prev_Current_Account_0.0,prev_Current_Account_1.0,Segmentation_01 - TOP,Segmentation_02 - PARTICULARES,Segmentation_03 - UNIVERSITARIO,Foreigner_Index_N,Foreigner_Index_S,...,prev_Payroll_Account_0.0,prev_Payroll_Account_1.0,prev_MAS_Particular_Account_0.0,prev_MAS_Particular_Account_1.0,prev_Home_Account_0.0,prev_Home_Account_1.0,prev_Particular_Plus_Account_0.0,prev_Particular_Plus_Account_1.0,prev_Derivada_Account_0.0,prev_Derivada_Account_1.0
0,0.0,1,0,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
1,0.0,1,0,1,0,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
2,0.0,1,0,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
3,0.0,1,0,0,1,0,1,0,1,0,...,1,0,1,0,1,0,0,1,1,0
4,0.0,1,0,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0


In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_numeric_np = scaler.fit_transform(df_numeric)
df_numeric = pd.DataFrame(normalized_numeric_np)
df_numeric.columns = ['Age','Customer_Seniority','Household_Income']

In [28]:
df_numeric = df_numeric.reset_index(drop=True)
df_categorical = df_categorical.reset_index(drop=True)
dates_and_cc = pd.DataFrame(train_data[['Date','Date_Acc_Open','Customer_Code']]).reset_index(drop=True)
dates_and_cc

,Date,Date_Acc_Open,Customer_Code
0,2015-02-28,2005-08-02,545090
1,2015-02-28,2005-08-30,545106
2,2015-02-28,2005-08-02,545102
3,2015-02-28,2005-08-02,545101
4,2015-02-28,2005-08-03,545158
5,2015-02-28,2005-08-02,545100
6,2015-02-28,2005-08-02,545099
7,2015-02-28,2005-08-02,545092
8,2015-02-28,2005-08-02,545091
9,2015-02-28,2005-08-02,545098


In [29]:
train_df_processed = pd.concat([dates_and_cc,df_numeric,df_categorical],axis=1)

In [30]:
train_df_processed

,Date,Date_Acc_Open,Customer_Code,Age,Customer_Seniority,Household_Income,New_customer_index,prev_Taxes_0.0,prev_Taxes_1.0,prev_Current_Account_0.0,...,prev_Payroll_Account_0.0,prev_Payroll_Account_1.0,prev_MAS_Particular_Account_0.0,prev_MAS_Particular_Account_1.0,prev_Home_Account_0.0,prev_Home_Account_1.0,prev_Particular_Plus_Account_0.0,prev_Particular_Plus_Account_1.0,prev_Derivada_Account_0.0,prev_Derivada_Account_1.0
0,2015-02-28,2005-08-02,545090,0.570175,0.487805,0.003309,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
1,2015-02-28,2005-08-30,545106,0.333333,0.487805,0.003733,0.0,1,0,1,...,1,0,1,0,1,0,1,0,1,0
2,2015-02-28,2005-08-02,545102,0.385965,0.487805,0.006799,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
3,2015-02-28,2005-08-02,545101,0.315789,0.487805,0.001312,0.0,1,0,0,...,1,0,1,0,1,0,0,1,1,0
4,2015-02-28,2005-08-03,545158,0.394737,0.439024,0.003071,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
5,2015-02-28,2005-08-02,545100,0.359649,0.487805,0.002641,0.0,1,0,1,...,1,0,1,0,1,0,1,0,1,0
6,2015-02-28,2005-08-02,545099,0.377193,0.487805,0.002130,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
7,2015-02-28,2005-08-02,545092,0.561404,0.487805,0.006206,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
8,2015-02-28,2005-08-02,545091,0.342105,0.487805,0.002762,0.0,1,0,1,...,0,1,1,0,1,0,1,0,1,0
9,2015-02-28,2005-08-02,545098,0.464912,0.487805,0.004816,0.0,1,0,0,...,1,0,1,0,1,0,1,0,1,0


In [32]:
train_df_processed.reset_index(drop=True,inplace=True)
y = train_data[y].reset_index(drop=True)

In [33]:
train_df_processed = pd.concat([train_df_processed,y],axis=1)

In [34]:
train_df_processed.isnull().sum()

Date                                0
Date_Acc_Open                       0
Customer_Code                       0
Age                                 0
Customer_Seniority                  0
Household_Income                    0
New_customer_index                  0
prev_Taxes_0.0                      0
prev_Taxes_1.0                      0
prev_Current_Account_0.0            0
prev_Current_Account_1.0            0
Segmentation_01 - TOP               0
Segmentation_02 - PARTICULARES      0
Segmentation_03 - UNIVERSITARIO     0
Foreigner_Index_N                   0
Foreigner_Index_S                   0
prev_Long_Term_Deposit_0.0          0
prev_Long_Term_Deposit_1.0          0
prev_Guarantees_0.0                 0
prev_Guarantees_1.0                 0
prev_Funds_0.0                      0
prev_Funds_1.0                      0
prev_Loans_0.0                      0
prev_Loans_1.0                      0
prev_Saving_Account_0.0             0
prev_Saving_Account_1.0             0
Sex_H       

In [35]:
# Save the data which is relevant, normalized and one-hot encoded
train_df_processed.to_pickle("train_month2_with_lag.pkl")

In [32]:
train_df_processed.columns

Index(['Date', 'Date_Acc_Open', 'Customer_Code', 'Age', 'Customer_Seniority',
       'Household_Income', 'New_customer_index', 'Activity_Index_0.0',
       'Activity_Index_1.0', 'Customer_Relation_Type_Beginning_Month_A',
       'Customer_Relation_Type_Beginning_Month_I',
       'Customer_Relation_Type_Beginning_Month_N',
       'Customer_Relation_Type_Beginning_Month_P',
       'Customer_Relation_Type_Beginning_Month_R', 'Province_Name_ALAVA',
       'Province_Name_ALBACETE', 'Province_Name_ALICANTE',
       'Province_Name_ALMERIA', 'Province_Name_ASTURIAS',
       'Province_Name_AVILA', 'Province_Name_BADAJOZ',
       'Province_Name_BALEARS, ILLES', 'Province_Name_BARCELONA',
       'Province_Name_BIZKAIA', 'Province_Name_BURGOS',
       'Province_Name_CACERES', 'Province_Name_CADIZ',
       'Province_Name_CANTABRIA', 'Province_Name_CASTELLON',
       'Province_Name_CEUTA', 'Province_Name_CIUDAD REAL',
       'Province_Name_CORDOBA', 'Province_Name_CORUÑA, A',
       'Province_Name_C